In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.3 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

In [3]:
data = load_dataset("rotten_tomatoes")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [4]:
from transformers import pipeline

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

pipe = pipeline(
    'text2text-generation',
    model = 'google/flan-t5-small',
    device = 'cuda:0'
)

In [5]:
# prepare our data
prompt = 'Is the following sentence positive or negative?'
data = data.map(lambda example: {'t5': prompt + example['text']})
data

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 1066
    })
})

In [6]:
# import tqdm and KeyDataset
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

In [8]:
from transformers import pipeline

In [12]:
pipe = pipeline(
    'text2text-generation',
    model = 'google/flan-t5-small',
    #device = 'cuda:0'
)

Device set to use cpu


In [14]:
y_pred = []
for output in tqdm(pipe(KeyDataset(data['test'], 't5')),
                   total = len(data['test'])):
                   text = output[0]['generated_text']
                   y_pred.append(0 if text == 'negative' else 1)

100%|██████████| 1066/1066 [02:16<00:00,  7.82it/s]


In [17]:
from sklearn.metrics import classification_report

In [20]:
def evalueate_performance(y_true, y_pred):
  performance = classification_report(y_true, y_pred,
                                   target_names=['Negative Review','Positive Review'])
  print(performance)

In [21]:
evalueate_performance(data['test']['label'], y_pred)

                 precision    recall  f1-score   support

Negative Review       0.83      0.84      0.83       533
Positive Review       0.84      0.83      0.83       533

       accuracy                           0.83      1066
      macro avg       0.83      0.83      0.83      1066
   weighted avg       0.83      0.83      0.83      1066



##ChatGPT for Classification

In [22]:
import openai

In [23]:
client = openai.OpenAI(api_key='Use open ai api key')

In [36]:
def chatgpt_generation(prompt, document, model="gpt-3.5-turbo-0125"):
  messages =[
      {'role': 'system',
       'content': 'You are a helpful assistant.'
        },
      {'role': 'user',
       'content': prompt.replace('[DOCUMENT]',  document)
        }

  ]

  chat_completion = client.chat.completions.create(
      messages = messages,
      model = model,
      temperature = 0
  )

  return chat_completion.choices[0].message.content

In [37]:
#Define a promp template as a base
prompt = """Predict whether the following document is a positive or negative
movie review:

[DOCUMENT]

If it is positive return 1 and if it is negative return 0. Do not give any other answer
"""

In [38]:
# predict the target using GPT
document = 'unpretentious, charming, quirky, original'
chatgpt_generation(prompt, document)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [39]:
# can be skip this if want to save (fees) credits
predictions = [
    chatgpt_generation(prompt, doc)for doc in tqdm(data['test']['text'])]

  0%|          | 0/1066 [00:00<?, ?it/s]

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [40]:
y_pred = [int(pred) for pred in predictions]
evalueate_performance(data['test']['label'], y_pred)

NameError: name 'predictions' is not defined